# Biological background
## Cellular biology
For a cell to enter mitosis, it first needs to undergo a series of changes called **interphase**. Interphase can be divided in three phases - G<sub>1</sub>, S, G<sub>2</sub>.
During the G<sub>1</sub> phase, amount of DNA in the cell remains the same. During the S phase, amount of DNA is increasing, only to be doubled in the G<sub>2</sub> phase.
Thus, depending on the amount of DNA, one can classify a single cell as belonging to either of those three phases. 

## DNA measuring
Content of the DNA can be measured indirectly, using intercalating flourescent dyes such as **propidium iodide** (PI) that bond to DNA molecules within the cells. Uniform suspension of cells is then passed through a flourescent cytometer that is capable of detecting single cells (*events*) and measuring the intensity of PI flourescence. Since this is an indirect measurement of DNA through PI signal intensity, repeated sampling of cells in say, G<sub>1</sub> phase, won't give the same values, but will give values that are distributed according to normal distribution. In general, measuring N amount of cells whose DNA content is X will be normally distributed with mean of PI(X) and standard deviation $\sigma$ where PI(X) is a function that translates DNA content into PI flourescence. Since we will be comparing cell content, we are not interested in the *exact amount* of DNA present in the cells, and thus we will make an assumption that PI(X) ~ X.

# Mathematical modelling
## Modeling DNA distributions
There are three aforementioned phases where the amount of DNA is different. We can model each of the phases by different function.
G<sub>1</sub> phase can be approximated as $ G_{1} \sim \mathcal{N}(\mu_{1},\,\sigma^{2}_{1})$ 

G<sub>2</sub> phase can be approximated as $ G_{2} \sim \mathcal{N}(\mu_{2},\,\sigma^{2}_{2})$

The problematic phase to model is the S phase, because the content of DNA in the phase is anywhere between the amount of DNA in G<sub>1</sub> and G<sub>2</sub> phase. The selected model for S phase will be an infinite sum of normal distributions according to the following:


$\int_{2n}^{4n} \!w_{x} \mathcal{N}(\mu_{x},\,\sigma^{2}_{x}) \, \mathrm{d}x$.

In here, 2n and 4n are the amount of DNA in G<sub>1</sub> and G<sub>2</sub> phase, while w represents a **weight** on the normal distribution, or the fraction of cells that have DNA content of X within the S phase. Making the assumption that all *weights* are equal, and that standard deviation are equal, and that $mu_{x}$ is simply x the above equation transforms into:

$\!w_{s}\int_{2n}^{4n}  \mathcal{N}(x,\,\sigma^{2}_{x}) \, \mathrm{d}x$.

The above infinite sum of Gaussians can be approximated by a uniform distribution:

$\int_{2n}^{4n}  \mathcal{N}(x,\,\sigma^{2}_{x}) \, \mathrm{d}x \sim U(2n,4n)$

## Mixture model

Standard operating procedure of a flow cytometer is to acquire a number of events (ranging in thousands) regardless of parameters of events. Thus, a suspension of cells that's a *mixture* of the cells from G<sub>1</sub>, S, G<sub>2</sub> phases is best approximated as a *mixture* model of DNA distributions of particular phases. We can write:


$model= w_{G1} \mathcal{N}(\mu_{1},\,\sigma^{2}_{1}) + w_{G2} \mathcal{N}(\mu_{2},\,\sigma^{2}_{2}) + (1-w_{G2}-w_{G1})\ U(mu_{1},mu_{2})$ 

Note that the expression $(1-w_{G2}-w_{G1})$ is equal to $w_{s}$, but since a mixture model requires all weights to be equal, we only need $n-1$ weights to describe a mixture of $n$ components. Standard constraints apply (ie. all weights must be equal or greater than 0, and their sum must be 1). 

## Application of cytostatic agent

Certain molecules can inhibit normal progression of cell division by various means. These molecules are called *cytostatics* (meaning they *stop* cells). Treating cells with a cytostatic will lower the amount of cells in S and/or G<sub>2</sub> phase. In our model, the *weights* of S and/or G<sub>2</sub> will lower by amounts $r_{1}$ and $r_{2}$, but the *weight* of G<sub>1</sub> will increase by those same amounts. We represent amounts $r_{1}$ and $r_{2}$ as one factor - *the cytostatic factor of difference* or **${d}$**. If ${d}$ is 100 (or 1, depending on implementation), all cells will be of G<sub>1</sub> phase and the model will only have only one relevant normal distribution.

This is also unlikely to happen, even with strong *cytostatics*, as there will be cells in S and/or G<sub>2</sub> phase prior to treating them with the cytostatic. Therefore we can add two additional parameters that *modify* the *strength* of acting cytostatic calling them ${gs}$ as "G strength" and ${ss}$ as "S strength". 

Implementing all three factors can be done by a formula:

${reduction}_{(G2)}=w_{(G2)} {d} {gs}$

${reduction}_{(S)}= w_{(S)} {d} {ss}$

$w_{(G2)} = w_{(G2)}-{reduction}_{(G2)} = w_{(G2)} (1-{d} {gs})$

$w_{(S)} = w_{(S)}-{reduction}_{(S)} = w_{(S)}(1-{d} {ss})$

$w_{(G1)} = w_{(G1)} + {reduction}_{(G2)} +  {reduction}_{(S)}$


In our model, *cytostatic factor d* represents our *certainty* that an applied agent is cytostatic. Respective strengths represent *power* of an applied cytostatic.

# Programing implementation

## Simulation of data

To simulate data, taking into consideration everything written above, we can write a Python code to *simulate* and *draw* the data. Additionally, interactive sliders for *cytostatic factor of difference*, *G strength*, and *S strength* is provided. Run the following code to get the simulated data!

In [47]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
plt.style.use('ggplot')


class SimulatedData():
    """
    .cf - cytostatic factor d that controls how much cells are "arrested" in cell cycle
    .jitter - controls how much cfd is applied (higher jitter, higher reduction)
    .sym - controls how symmetric is cf at taking from G2 or S phase, bigger sym favors G2 phase, smaller favors S phase
    .data - data
    .parameters - parameters of distribution
    
    """
    def __init__(self,cfd,gs=90,ss=90,seed=0,size=15000):
        """
        CFD - cytostatic factor of difference
        Controls how much cells are "arrested" in cell cycle
        
        GS - G strength
        Controls how much cfd takes from G2 phase
        
        SS - S strength
        Controls how much cfd takes from S phase
        
        """
        if cfd<0 or cfd>100:
            raise ValueError("cfd must be between 0 and 100")
        if gs<0 or gs>100:
            raise ValueError("gs must be between 0 and 100")
        if ss<0 or ss>100:
            raise ValueError("ss must be between 0 and 100")
        
        np.random.seed(seed)
        
        #generate the mean1, mean2
        m1=np.random.randint(100,300)+np.random.sample()
        m2=2*m1+20*np.random.sample()
        
        #generate the sd1, sd2
        s1=0.1*m1+np.random.sample()
        s2=0.1*m2+np.random.sample()
        
        #generate the weights
        w1=np.random.uniform(0.3,0.5)
        w2=np.random.uniform(0.3,0.5)
        w3=1-w1-w2
        
        #apply the factor
        if cfd>0:
            cf=cfd/100
            g=gs/100
            s=ss/100
            a=w2*cf*g
            b=w3*cf*s
            #lowering weights
            w2=w2-a
            w3=w3-b
            #adding weights
            w1=w1+a+b 

        #G0 phase (2n)
        data1=np.random.normal(loc=m1,scale=s1,size=int(w1*size))
        #G2 phase (4n)
        data2=np.random.normal(loc=m2,scale=s2,size=int(w2*size))
        #S phase (2n-4n)
        data3=np.random.uniform(low=m1,high=m2,size=int(w3*size))
        #concat data
        data=np.concatenate([data1,data2,data3])
        param_dic={"m1":m1,"m2":m2,"s1":s1,"s2":s2,"w1":w1,"w2":w2}
        self.data=data
        self.parameters=param_dic
        self.cf=cfd
        self.gs=gs
        self.ss=ss
        

%matplotlib inline

def update_plot(cfd,gs,ss):
    new_data=SimulatedData(cfd,gs,ss)
    data=new_data.data
    hist_plot=sns.histplot(data,stat="density",kde=True)

cfd=widgets.IntSlider(min=0, max=100, step=1, value=0, description ="Cytostatic")
gs=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="G strength")
ss=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="S strength")
widgets.interactive(update_plot,cfd=cfd,gs=gs,ss=ss)

interactive(children=(IntSlider(value=0, description='Cytostatic'), IntSlider(value=75, description='G strengt…

## Parameter estimation

Recalling our model:

$model= w_{G1} \mathcal{N}(\mu_{1},\,\sigma^{2}_{1}) + w_{G2} \mathcal{N}(\mu_{2},\,\sigma^{2}_{2}) + (1-w_{G2}-w_{G1})\ U(\mu_{1},\mu_{2})$ 

We can see that the model has 6 parameters - 3 for each Gaussian. Question is, if we have the data, can we recover those parameters? 
We will "store" our parameters into one vector, **$\theta$**, and consider the case of one point (one cell, one event).

$m(x_{i} | \theta)=w_{G1} \mathcal{N}(x_{i}|\mu_{1},\,\sigma^{2}_{1}) + w_{G2} \mathcal{N}(x_{i}|\mu_{2},\,\sigma^{2}_{2}|x_{i}) + (1-w_{G2}-w_{G1})\ U(x_{i}|\mu_{1},\mu_{2})$ 

Every event is independent from each other, and since the above mentioned model represents a kind of a *probability* distribution, for many measurements $N$ stored in a vector $X$, we can write

$m(X | \theta)=\prod_n m(x_{i} | \theta)$
$=\prod_n\ [w_{G1} \mathcal{N}(x_{i}|\mu_{1},\,\sigma^{2}_{1}) + w_{G2} \mathcal{N}(x_{i}|\mu_{2},\,\sigma^{2}_{2}|x_{i}) + (1-w_{G2}-w_{G1})\ U(x_{i}|\mu_{1},\mu_{2})]$ 

The above expression is rather unwieldy, as probabilities are very small, and their product will likewise be very small. To ease the calculations, we apply log transform. Log transform will also transform multiplication to summation. We will take the **negative** log transform, because the problem will then be one of minimization, rather than maximization.

$\mathcal{L}(\theta)=-log m(X|\theta)=-\sum log m(x_{i}|\theta)$

The above function will be the **objective** function for our model given our data. Thus, problem of finding optimal parameters can is one of minimization of the above function.

There are following *constraints* we need to take in consideration. First, all our parameters must be positive (deviations and means must be strictly positive), especially our weights, and the sum of our weights must be less than or equal to 1. Luckily, those constraints are *linear*, and can be expressed in the form of a dot product of a matrix **A** and parameter vector **$\theta$** We define the matrix **A** and parameter vector **$\theta$** below:

$A=\begin{bmatrix}              
1 & 1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}$

$\theta=\begin{bmatrix}
w_{1}\\
w_{2}\\
\mu_{1}\\
\mu_{2}\\
\sigma_{1}\\
\sigma_{2}\\
\end{bmatrix}$

We can define **lower boundary** to be a zero vector, and **upper boundary** to be a vector with first value 1, and the rest infinity. We'll use @ symbol to denote matrix multiplication for concordance with numpy operator.

**0** $\leq A @ \theta \leq [1 \infty \infty \infty \infty \infty \infty]^{T}$

Lastly, this needs to be implemented in Python. We will make extensive use of autograd library to calculate gradient (and value!) of our objective function and of scipy.optimize library to minimize both gradient and value of our function, as well as using LinearConstraint class so our minimization can converge without issues. 

Finally, since this is a simulated experiment all our prior parameters are known! It is then possible to define error as (original-determined)/(original) in percentage, and we can then average the vector. Code below does just that. It simulated the data according to the user provided input (just like in plotting), recovers the original parameters, determines new parameters according to the above equations (functions optimal_parameter_finding and neglog), and then reports the discrepancy between the as average error.

*Note: Since autograd and scipy.minimize are algorithmic, they can ocassionally give errors and warning. Since we're using logarithmic transformation, if we take log of anything lesser than 0, we'll get an error. These errors are harmless, and both autograd and scipy.minimize will gloss over those errors and they won't affect end result. Nevertheless, for aesthetic purposes, we elect to suppress error warnings*


In [55]:
import warnings
warnings.filterwarnings('ignore') #suppress error warnings

from autograd.scipy.stats import norm
from autograd import numpy as np
from autograd import value_and_grad
from scipy.optimize import LinearConstraint as lincon
from scipy.optimize import minimize
from scipy.stats import uniform

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)}) #<-we dont want to get bogged down in a million of decimal points

def neglog(params,data):
    """
    order is w1,w2,m1,m2,s1,s2
    """
    #parameter unpacking
    w1,w2,m1,m2,s1,s2=params        
    gaus=w1*norm.pdf(data,m1,s1)+(w2)*norm.pdf(data,m2,s2)

    loc=min(m1._value,m2._value)
    scale=abs(m1._value-m2._value)
        
    unif=(1-w1-w2)*uniform.pdf(data,loc,scale)
 
    imn=gaus+unif
    imn=np.log(imn)
    imn=-1*imn.sum()
    return(imn)

def optimal_parameter_finding(data):
    #define our constraints
    A=np.hstack((np.array([1,1,0,0,0,0]).reshape(6,1),np.identity(6))).T
    lb=np.zeros(7)
    up=np.full((7),np.inf)
    up[0]=1
    
    conmat=lincon(A,lb,up)
    
    results = minimize(
                value_and_grad(neglog), # see autograd docs.
                x0 = np.array([0.4, 0.4, 200, 500, 30,50]), # initial value
                args=(data,), #additional argument, namely data, must be passed
                jac=True, #this tells the algorithm that our function returns (value(x),gradient(x))
                constraints=conmat #these are our contraints
                )
        
    return(results.x)

def data_drawer(original_data,det_parameters):
    size=15000
    #unpacking determined parameters so we can redraw our data
    w1=det_parameters[0]
    w2=det_parameters[1]
    w3=1-w1-w2
    m1=det_parameters[2]
    m2=det_parameters[3]
    s1=det_parameters[4]
    s2=det_parameters[5]
    #G0 phase (2n)
    data1=np.random.normal(loc=m1,scale=s1,size=int(w1*size))
    #G2 phase (4n)
    data2=np.random.normal(loc=m2,scale=s2,size=int(w2*size))
    #S phase (2n-4n)
    data3=np.random.uniform(low=m1,high=m2,size=int(w3*size))
    #concat data
    det_data=np.concatenate([data1,data2,data3])
    hist_plot1=sns.histplot(original_data,stat="density",kde=True,color="blue",alpha=0.5)
    hist_plot2=sns.histplot(det_data,stat="density",kde=True,color="red",alpha=0.5)
    

def simulate_new_data(cfd,gs,ss):
    simulated_data=SimulatedData(cfd,gs,ss)
    data=simulated_data.data
    parameters=simulated_data.parameters
    original_parameters=[parameters["w1"],parameters["w2"],parameters["m1"],parameters["m2"],parameters["s1"],parameters["s2"]]
    original_parameters=np.array(original_parameters,dtype=float)
    
    determined_parameters=optimal_parameter_finding(data)
    error_v=(original_parameters-determined_parameters)/original_parameters*100
    print("Discrepancy between original and determined is: ",error_v," %")
    print("Average error is:",round(error_v.mean(),2)," %")
    #plot the difference
    print("Below is the plot of original data in blue, and data reconstructed from determined parameters in red")
    data_drawer(data,determined_parameters)

cfd=widgets.IntSlider(min=0, max=100, step=1, value=65, description ="Cytostatic")
gs=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="G strength")
ss=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="S strength")
widgets.interactive(simulate_new_data,cfd=cfd,gs=gs,ss=ss)



interactive(children=(IntSlider(value=65, description='Cytostatic'), IntSlider(value=75, description='G streng…

Our simulated experiment will include one sample where there was no cytostatic (cfd=0), and one sample where a cytostatic is present (cfd=100). Our question is, if we have those two samples, can we determine G and S strength?

In [57]:
def double_simulation(gs,ss):
    noCancer=SimulatedData(0,gs,ss)
    Cancer=SimulatedData(100,gs,ss)
    
    ##TAKING STRENGTHS DIRECTLY
    #weights of cfd=0
    w1_0=noCancer.parameters["w1"]
    w2_0=noCancer.parameters["w2"]
    w3_0=1-w1_0-w2_0
    
    #weights of cfd=100
    w1_100=Cancer.parameters["w1"]
    w2_100=Cancer.parameters["w2"]
    w3_100=1-w1_100-w2_100
    
    #storing strengths
    g_str_orig=(w2_0-w2_100)/(w2_0)
    s_str_orig=(w3_0-w3_100)/(w3_0)
    
    ##TAKING STRENGTHS FROM PARAMETER ESTIMATION
    par_0=optimal_parameter_finding(noCancer.data)
    par_100=optimal_parameter_finding(Cancer.data)
    
    #weights of cfd=0
    w1_0=par_0[0]
    w2_0=par_0[1]
    w3_0=1-w1_0-w2_0
    
    #weights of cfd=100
    w1_100=par_100[0]
    w2_100=par_100[1]
    w3_100=1-w1_100-w2_100
    
    #recovering the determined strengths
    g_str_det=(w2_0-w2_100)/w2_0
    s_str_det=(w3_0-w3_100)/w3_0

    print(
        "Recovered strengths are: G=",
          round(g_str_orig*100,2),
          " S=",
          round(s_str_orig*100,2)
         )
          
    print(
        "Estimated strengths are: G=",
        round(g_str_det*100,2),
        " S=",
        round(s_str_det*100,2)
        )
    print(
        "Error is G(err)=",
        round((g_str_orig-g_str_det)/(g_str_orig)*100,2),
        "%, S(err)=",
        round((s_str_orig-s_str_det)/(s_str_orig)*100,2),
        "%")
   
    
    
    
gs=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="G strength")
ss=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="S strength")
widgets.interactive(double_simulation,gs=gs,ss=ss)

interactive(children=(IntSlider(value=75, description='G strength'), IntSlider(value=75, description='S streng…

The above answer is yes, we can! Generally speaking, when doing an experiment, the scientist will have to have one negative control (where there is no cytotoxic substance present) and one positive control (where there is a substance they are positively sure is cytotoxic), as well as the actual experiment. In the actual experiment, cytotoxicity of the substance is not known, and both negative and positive control are here to compare the tested substance against no substance and cytotoxic substance.

## Estimating cytotoxicity of a substance 
In our simulated example, we demonstrated that we can recover G and S strengths when we have negative control (corresponding to ${d=0}$) and positive control (corresponding to ${d=100}$). 
Our next question will be that if we know G and S strengths, and a substance we don't know for sure is cytotoxic (corresponding to ${d=x}$), can we determine $x$?

Before simulating further data, we'll bring up our mathematical model again (we replaced $1-w_{G1}-w_{G2}$ with $w_{S}$ to make calculation easier):

$\mathcal{L}(\theta)=-log m(X|\theta)=-\sum log m(x_{i}|\theta)$

$m(x_{i} | \theta)= [w_{G1} \mathcal{N}(x_{i}|\mu_{1},\,\sigma^{2}_{1}) + w_{G2} \mathcal{N}(x_{i}|\mu_{2},\,\sigma^{2}_{2}|x_{i}) + w_{S}\ U(x_{i}|\mu_{1},\mu_{2})]$ 

Notice that this model makes no notice of parameters we put up on sliders (G, S strengths, cytostatic factor). Indeed, these parameters are modeled to act upon the weights, and only on weights, in a non-linear fashion that ultimately returns a value that is apt for a weight value - greater than or equal to 0, and sum of all weights ultimately being 1:

$w_{i}=w_{i} (1- {s_{i}}  {d}) $  or $w_{j}=w_{j} + \sum({s_{i}}  {d}) $

Our model above models treats one population treated with nothing and another population treated with something as two different things. In order to better glean at the effects of factors that reduce (n-1) weights and increase one weight, we need to find modify our model with additional parameters - namely strengths, and the cytotoxic factor d. This amounts to plugging the above equation in our model, and extending $\theta$ vector with additional parameters. Ours is a model of 3 components (2 gaussian, 1 uniform), and thus has 2 strengths and 1 factor - we extend our $\theta$ by three additional parameters!

$m(x_{i} | \theta_{ext})= [(w_{G1}+w_{G2}{s_{2}}{d}+w_{S}{s_{3}}{d}) \mathcal{N}(x_{i}|\mu_{1},\,\sigma^{2}_{1}) + w_{G2}(1-{s_{2}}  {d}) \mathcal{N}(x_{i}|\mu_{2},\,\sigma^{2}_{2}|x_{i}) + w_{S}(1-{s_{2}}  {d})\ U(x_{i}|\mu_{1},\mu_{2})]$ 

$\theta_{ext}=\begin{bmatrix}
w_{1}\\
w_{2}\\
\mu_{1}\\
\mu_{2}\\
\sigma_{1}\\
\sigma_{2}\\
d\\
s_{1}\\
s_{2}\\
\end{bmatrix}$

In case where d is equal to 0, strengths are simply irrelevant. What can the strengths represent? They could be a fraction of the cells that are in G<sub>2</sub> or S phase right up until a moment a perfect cytostatic (a theoretical agent that stops cell cycle completely) enters the cell mixture - they can represent a threshold of weights of G<sub>2</sub> and S phase components that the respective weights cannot go lower than.

Our $\theta_{ext}$ vector has 9 parameters, and our experiment with positive and negative control enables us to *determine* all 9 parameters. Assuming that the strengths remain constant within different experiments, that they're a property solely of a particular population of cells rather than a function of various other elements, our extended model for an unknown ${d=x}$ will only have one parameter to be deterimened, and that is ${d}$!

Rewriting our model differently:
    
$m_{unk}(x_{i} | {d})= [(w_{G1}+w_{G2}{s_{2}}{d}+w_{S}{s_{3}}{d}) \mathcal{N}(x_{i}|\mu_{1},\,\sigma^{2}_{1}) + w_{G2}(1-{s_{2}}  {d}) \mathcal{N}(x_{i}|\mu_{2},\,\sigma^{2}_{2}|x_{i}) + w_{S}(1-{s_{2}}  {d})\ U(x_{i}|\mu_{1},\mu_{2})]$  
    
$\mathcal{L}({d})=-log m_{unk}(X|{d})=-\sum log m_{unk}(x_{i}|{d})$

We can consider all other parameters, other than d, as constants. The optimal parameter d will be the one that minimizes the above function.

The following code will do the following things

1. Take user provided G and S parameters and simulate two datasets corresponding to positive (D=100) and negative (D=0) control
2. Estimate parameters of dataset according to the "optimal_parameter_finding" function
3. Estimate G and S parameters from optimal parameters for positive (D=100) and negative (D=0) control
4. Take user provided D parameter (corresponding to cytotoxicity of a substance) and simulate one dataset.
5. Estimate the D parameter according to the above mentioned idea.
6. Draw all our experiments, positive and negative control, as well as tested substance

G and S parameters are considered to be a function of cell population used in the experiment and are applied to all experiments equally. Cytostatic slider controls "cytotoxicity" of the supstance used in experiment. 

Our sucess will be if we can recover user inputed value for Cytostatic to a reasonable degree of precision.
    


In [58]:
def estimate_strengths(posPar,negPar):
    #weights of cfd=0
    w1_0=negPar[0]
    w2_0=negPar[1]
    w3_0=1-w1_0-w2_0
    
    #weights of cfd=100
    w1_100=posPar[0]
    w2_100=posPar[1]
    w3_100=1-w1_100-w2_100
    
    #recovering the determined strengths
    g_str_det=(w2_0-w2_100)/w2_0
    s_str_det=(w3_0-w3_100)/w3_0
    return(g_str_det,s_str_det)

def neglog_cytostat(d,data):
    """
    constants are in order w1,w2,m1,m2,s1,s2,gstr,sstr
    """
    #parameter unpacking
    w1,w2,m1,m2,s1,s2,gstr,sstr=constants
    ws=1-w1-w2
    
    #applying strengths
    w2=w2*(1-d*gstr)
    ws=ws*(1-d*sstr)
    w1=1-w2-ws #going about it a roundabout way
    
    
    gaus=w1*norm.pdf(data,m1,s1)+(w2)*norm.pdf(data,m2,s2)

    loc=min(m1,m2)
    scale=abs(m1-m2)
    unif=ws*uniform.pdf(data,loc,scale)
 
    imn=gaus+unif
    imn=np.log(imn)
    imn=-1*imn.sum()
    return(imn)

def cytostat_param_finding(exData,constants):
    
    results = minimize(
                value_and_grad(neglog_cytostat), # see autograd docs.
                x0 = 0.5, # initial value
                args=(exData,), #additional argument, namely data, must be passed
                jac=True, #this tells the algorithm that our function returns (value(x),gradient(x))
                bounds= ((0,1),) #d must be between 0 and 1
                )
    return(results)
    
def drawing_our_experiment(posData,negData,exData,cfd):
    #transfer our data into pandas DataFrame
    
    pos=pd.DataFrame(posData,columns=["DNA content"])
    pos.insert(0,"Cytostatic",100)
    
    neg=pd.DataFrame(negData,columns=["DNA content"])
    neg.insert(0,"Cytostatic",0)
    
    ex=pd.DataFrame(exData,columns=["DNA content"])
    ex.insert(0,"Cytostatic",cfd)
    
    dataset=pd.concat([neg,ex,pos])
    sns.displot(data=dataset,x="DNA content",hue="Cytostatic",col="Cytostatic",stat="density",kde=True)
    
def main_function(cfd,gs,ss):
    #first we simulate our controls
    posData=SimulatedData(100,gs,ss)
    negData=SimulatedData(0,gs,ss)
    #then we estimate parameters of controls
    posPar=optimal_parameter_finding(posData.data)
    negPar=optimal_parameter_finding(negData.data)
    #then we estimate strengths
    gstr,sstr=estimate_strengths(posPar,negPar)
    #print(gstr,sstr)
    
    #we will pack our estimated parameters (weights, means, stdevs, strengths) into one array called constant
    #the constants are in order (w1,w2,m1,m2,s1,s2,gstr,sstr)
    global constants #we make constants global because Autograd wrapped numpy doesn't behave
    constants=np.append(negPar,[gstr,sstr])
    
    #now we will simulate our "experimental dataset"
    exData=SimulatedData(cfd,gs,ss)
    
    #we will now draw our datasets
    drawing_our_experiment(posData.data,negData.data,exData.data,cfd)
    
    #and finally we find try to recover our parameter
    result=cytostat_param_finding(exData.data,constants)
    d=result.x
    print("Your Cytostatic parameter is ",d*100)
    


cfd=widgets.IntSlider(min=1, max=99, step=1, value=50, description ="Cytostatic")
gs=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="G strength")
ss=widgets.IntSlider(min=0, max=100, step=1, value=75, description ="S strength")
widgets.interactive(main_function,cfd=cfd,gs=gs,ss=ss)

interactive(children=(IntSlider(value=50, description='Cytostatic', max=99, min=1), IntSlider(value=75, descri…